<h1 style="text-align: center;"> Losers Queue Analysis </h1>

<p>This notebook contains code used extract match information from Riot's API and analyse it, with the goal of better understanding the League of Legends Matchmaking system and if there are any evidence of the existence of a losers queue</p>


In [23]:
from riotwatcher import LolWatcher, ApiError
# import pandas as pd #for some reason pypy isnt able to install pandas so im not using it for now
import json
import time

# stored in a different file so I dont accidentally upload my
riot_api_key = open("riot_api_key.txt", "r")
# api key to git
watcher = LolWatcher(riot_api_key.read())
my_region = 'euw1'


def downloadChallengerIds(region, queue):
    '''
        Downloads all 300 challenger account ids from the desired queue and returns a list containing all
        of those.
    '''
    list_of_challengers = watcher.league.challenger_by_queue(my_region, queue)
    challenger_accountIds = []
    for challenger in list_of_challengers['entries']:
        summoner = watcher.summoner.by_id(
            my_region, challenger['summonerId'])   # we do this step because
        # the challenger_by_queue method does not return AccountId information. In order to get the accountId
        # we need to call the summoner.by_id() method and get the accountID returned.
        # I can only make a limited amount of requests per 2 minutes, so im using this to not get
        time.sleep(1)
        # an error
        challenger_accountIds.append(summoner['accountId'])
    return challenger_accountIds

# challenger_accountIds = downloadChallengerIds(my_region, 'RANKED_SOLO_5x5')
# with open('challenger_account_ids.json', 'w') as f:  # saves the accountIds to disk
#     json.dump(challenger_accountIds, f)


with open('challenger_account_ids.json') as f:  # reads the chall account ids saved previously
    challenger_accountIds = json.load(f)


def downloadMatches(region, accountId):
    '''
        Returns a list with all last 100 matches played by the accountId
    '''
    detailed_matches_list = []
    matches = watcher.match.matchlist_by_account(
        region, accountId)  # gets all the 100 matches ids
    for match in matches['matches']:
        # waits for a sec so we don't surpass the max amount of requests per minute
        time.sleep(1)
        start = time.time()
        match_detail = watcher.match.by_id(region, match['gameId'])
        print(f'It took {time.time() - start} seconds to request the match')
        start = time.time()
        match_detail["_id"] = match_detail["gameId"] # this will make filtering easier in mongoDB later
        detailed_matches_list.append(match_detail)
        print(
            f'It took {time.time() - start} seconds to append the match to the list')
    return detailed_matches_list


In [24]:
# Creating a MongoDB to store match information
import pymongo

dbclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = dbclient["losers_queue"]
matches = mydb['matches']

It took 0.2762300968170166 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.2672841548919678 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.27230405807495117 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.37502193450927734 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.27723002433776855 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.29221296310424805 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.27229881286621094 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.5505580902099609 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.31316304206848145 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.266261816024

It took 0.2792520523071289 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.31893014907836914 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.2682828903198242 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.267287015914917 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.3450791835784912 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.30733609199523926 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.3191211223602295 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.26927995681762695 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.36099982261657715 seconds to request the match
It took 0.0 seconds to append the match to the list
It took 0.288229942321777

HTTPError: 504 Server Error: Gateway Timeout for url: https://euw1.api.riotgames.com/lol/match/v4/matches/5143938017

In [ ]:
# for each challenger ID, gets information about 100 matches and put them inside mongoDB 'matches' 
# collection
count = 0
for challengerId in challenger_accountIds[count:5]: 
    time.sleep(1)
    matches.insert_many(downloadMatches(my_region, challengerId)) 
    count += 1
    print(count)
    


<p> Now that we have added all the match information from the 300 top ranked challenger players, we need to analyse them.  </p>

In [28]:
# Reading the matches from the collection

summoner = "Lorde Poeira"
query = { "participantIdentities.player.summonerName:" f"{summoner}"}
for match in matches.find():
  print(match)


    
# def lastMatch():
#     '''
#         Asks for a summoner name and tells you if that summoner has won its last match or not
#     '''
#     summonerName = input('Type your summoner name: ').casefold()
#     try:
#         summoner = watcher.summoner.by_name(my_region, summonerName)
#     except ApiError as err:
#         if err.response.status_code == 429:
#             print('this retry-after is handled by default by the RiotWatcher library')
#             print('future requests wait until the retry-after time passes')
#         elif err.response.status_code == 404:
#             print('Summoner with that name not found.')
#             raise
#         else:
#             raise

#     # all objects are returned (by default) as a dict

#     # my_ranked_stats = watcher.league.by_summoner(my_region, summoner['id'])
#     # # print(my_ranked_stats)

#     my_matches = watcher.match.matchlist_by_account(my_region, summoner['accountId'])


#     # fetch last match detail
#     last_match = my_matches['matches'][0]
#     match_detail = watcher.match.by_id(my_region, last_match['gameId'])

#     participants = []
#     for row in range(0, 10):
#         participants_row = {'win': match_detail['participants'][row]['stats']['win'],
#                             'SummonerName': match_detail['participantIdentities'][row]['player']['summonerName'].casefold(),
#                             }
#         participants.append(participants_row)


#     df = pd.DataFrame(participants)

#     if not df['win'][df['SummonerName'] == summonerName].empty:
#         if df['win'][df['SummonerName'] == summonerName].bool():
#             print('You won your last match.')
#         else:
#             print('You lost your last match')
#     else:
#         print('Couldn\'t find information about your last match')


{'_id': 5153932981, 'gameId': 5153932981, 'platformId': 'EUW1', 'gameCreation': 1615893661995, 'gameDuration': 1846, 'queueId': 420, 'mapId': 11, 'seasonId': 13, 'gameVersion': '11.5.361.3108', 'gameMode': 'CLASSIC', 'gameType': 'MATCHED_GAME', 'teams': [{'teamId': 100, 'win': 'Fail', 'firstBlood': True, 'firstTower': True, 'firstInhibitor': False, 'firstBaron': False, 'firstDragon': False, 'firstRiftHerald': True, 'towerKills': 5, 'inhibitorKills': 0, 'baronKills': 0, 'dragonKills': 1, 'vilemawKills': 0, 'riftHeraldKills': 2, 'dominionVictoryScore': 0, 'bans': [{'championId': 236, 'pickTurn': 1}, {'championId': 77, 'pickTurn': 2}, {'championId': -1, 'pickTurn': 3}, {'championId': 234, 'pickTurn': 4}, {'championId': 246, 'pickTurn': 5}]}, {'teamId': 200, 'win': 'Win', 'firstBlood': False, 'firstTower': False, 'firstInhibitor': True, 'firstBaron': True, 'firstDragon': True, 'firstRiftHerald': False, 'towerKills': 9, 'inhibitorKills': 2, 'baronKills': 1, 'dragonKills': 3, 'vilemawKills':

{'_id': 5128709339, 'gameId': 5128709339, 'platformId': 'EUW1', 'gameCreation': 1614771596280, 'gameDuration': 2009, 'queueId': 420, 'mapId': 11, 'seasonId': 13, 'gameVersion': '11.5.361.3108', 'gameMode': 'CLASSIC', 'gameType': 'MATCHED_GAME', 'teams': [{'teamId': 100, 'win': 'Win', 'firstBlood': True, 'firstTower': False, 'firstInhibitor': True, 'firstBaron': True, 'firstDragon': False, 'firstRiftHerald': True, 'towerKills': 11, 'inhibitorKills': 3, 'baronKills': 1, 'dragonKills': 3, 'vilemawKills': 0, 'riftHeraldKills': 1, 'dominionVictoryScore': 0, 'bans': [{'championId': 526, 'pickTurn': 1}, {'championId': 77, 'pickTurn': 2}, {'championId': 107, 'pickTurn': 3}, {'championId': 91, 'pickTurn': 4}, {'championId': 114, 'pickTurn': 5}]}, {'teamId': 200, 'win': 'Fail', 'firstBlood': False, 'firstTower': True, 'firstInhibitor': False, 'firstBaron': False, 'firstDragon': True, 'firstRiftHerald': False, 'towerKills': 3, 'inhibitorKills': 0, 'baronKills': 0, 'dragonKills': 2, 'vilemawKills'